In [1]:
import time
# import IPython
import os
import json
from datetime import datetime
import pyswarm
import pandas

import joblib
import numpy as np
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
from tensorflow import keras
import tensorflow.keras.backend as K
from pyswarm import pso


from keras.models import Sequential, Model
# from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda, Layer
# from tensorflow.keras import losses
# from keras.utils.vis_utils import plot_model, model_to_dot

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import sklearn.preprocessing

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error

K.set_floatx('float32')

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

tf.enable_v2_behavior()

from tester.tester import Tester

import random


2022-03-24 18:29:13.717773: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-24 18:29:13.717808: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
result_idx=1

In [157]:
lb = [2.2, 1.08, -40.]  # lower bounds of the design variables, i.e. input params for the sut_model
ub = [2.8, 1.41, 175.] 
# log_path = os.path.join('.','2022_02_24_17_12_27_inf_gain')
# log_path = os.path.join('.','2022_02_24_19_52_58_inf_gain')
log_path = os.path.join('.','Experiments','bay_opt_min'+str(result_idx))


save_path= os.path.join('.','BO_MIN_data')

In [158]:
path = os.path.join(log_path, 'run_stats.txt')
with open(path, 'r') as file:
    run_stats = json.load(file)    

    
num_samples = run_stats['num_samples']
training_proportion = run_stats['training_proportion']
optimization_proportion = 1. - training_proportion

log_std_coefficient = run_stats['log_std_coefficient']
constant_std_coefficient = run_stats['constant_std_coefficient']

run_stats

{'num_samples': 100,
 'training_proportion': 0.25,
 'log_std_coefficient': 4.0,
 'constant_std_coefficient': 10.0,
 'seed': 2109997854}

In [159]:
path = os.path.join(log_path, 'training.npyz')
with open(path, 'rb') as file:
    training_data = np.load(file)
    training_Xs=training_data['points']
    training_ys=training_data['values']
    norm_training_Xs=training_data['norm_points']
    norm_training_ys=training_data['norm_values']
    lls_=training_data['loss']
    amplitude= training_data['amplitude']
    relevances= training_data['relevances']
    noise_var= training_data['noise']
    

In [160]:
path=os.path.join(log_path,'scaler_X.pkl')
transform_X = joblib.load(path)
path=os.path.join(log_path,'scaler_y.pkl')
transform_y = joblib.load(path)


norm_ub = transform_X.transform([ub])[0].astype(np.float32)
norm_lb = transform_X.transform([lb])[0].astype(np.float32)

/home/llarrauriborroto/Documents/Projects/advanced-bayesian-opt/lib/python3.8/site-packages/sklearn/base.py:313: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.2 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [161]:
norm_training_Xs


array([[-1.140739  , -1.1259642 , -0.3972631 ],
       [ 0.3843126 ,  1.141078  , -0.3972631 ],
       [-0.3782126 ,  0.00755684, -0.8276846 ],
       [ 0.7909939 , -1.5038043 ,  1.7229613 ],
       [ 1.5535191 ,  0.00755684, -0.19002312],
       [-1.4965842 , -1.1259642 , -0.8276846 ],
       [-1.4965842 ,  0.76323795, -0.8276846 ],
       [-1.140739  , -0.74812424, -1.4653461 ],
       [ 0.7909939 , -1.1259642 , -0.8276846 ],
       [ 1.146839  , -0.74812424, -0.19002312],
       [ 0.7909939 ,  1.6133778 ,  1.9620844 ],
       [-0.3782126 , -1.1259642 ,  0.44763836],
       [-0.7340589 ,  1.141078  ,  0.7664691 ],
       [ 1.146839  ,  1.6133778 , -0.19002312],
       [-0.3782126 , -0.74812424,  1.0852998 ],
       [-1.4965842 ,  1.141078  ,  1.4041306 ],
       [ 1.146839  ,  0.00755684, -0.3972631 ],
       [ 1.146839  ,  0.76323795, -1.1465153 ],
       [ 0.3843126 ,  0.3853968 ,  0.12880762],
       [-1.4965842 , -1.5038043 , -0.3972631 ],
       [-0.3782126 , -0.74812424,  0.766

In [162]:
kernel = tfk.ExponentiatedQuadratic(amplitude, length_scale=1.0)
kernel = tfk.FeatureScaled(kernel, scale_diag=relevances)
gp_model = tfd.GaussianProcessRegressionModel(
    kernel=kernel,
    index_points=[[1.2353002 , -0.7703848 ,  1.1779659]],
    observation_index_points=norm_training_Xs,
    observations=norm_training_ys,
    observation_noise_variance=noise_var,
    predictive_noise_variance=0.,
    jitter=1e-4)

gp_model.mean()

<tf.Tensor: shape=(), dtype=float32, numpy=-1.0479727>

In [163]:
path = os.path.join(log_path, 'optimization_rounds.npyz')
with open(path, 'rb') as file:
    optimization_data = np.load(file)
    real_values=optimization_data['values']
    real_choices=optimization_data['choices']
    mean_regrets=optimization_data['mean_regrets']


In [164]:
values= transform_y.transform(real_values.reshape(-1,1))
choices= np.array(transform_X.transform(real_choices), dtype=np.float32)
values=np.array(values.flatten(),dtype=np.float32)


In [165]:
filepath = os.path.join(log_path, 'validation.npyz')


with open(filepath, 'rb') as file:
    data = np.load(file)
    valid_real_Xs=data['points']
    valid_real_ys=data['values']


In [166]:
valid_norm_Xs=np.array(transform_X.transform(valid_real_Xs),dtype=np.float32)
valid_norm_Xs
print(valid_real_ys)

[1.1364679 1.1364533 1.1364809 1.1364826 1.1364546 1.136453  1.1364746
 1.1364664 1.1364375 1.1366153 1.1366072 1.1366217 1.1366186 1.1366061
 1.1365982 1.1366205 1.1366148 1.1366053 1.1367441 1.136754  1.1367435
 1.1367403 1.1367362 1.1367447 1.1367383 1.1368902 1.1368879 1.1368802
 1.1368752 1.1368766 1.1368825 1.1368804 1.1368675 1.1369122 1.1369243
 1.1369307 1.1369127 1.1369225 1.1369193 1.1369181 1.1369164 1.1369104
 1.1369476 1.1369559 1.1369513 1.136933  1.1369461 1.1369497 1.1369456
 1.1369486 1.1369312 1.1369246 1.1369195 1.1369295 1.1369048 1.1369209
 1.1369205 1.1369218 1.1369188 1.1368982 1.1368333 1.1368339 1.1368332
 1.1368451 1.1368332 1.1368301 1.1368381 1.1368384 1.1368365 1.1366992
 1.136702  1.1366899 1.136702  1.1366932 1.1366885 1.1366879 1.1366869
 1.1366874 1.1364846 1.1365002 1.1364875 1.1364605 1.1364737 1.1364778
 1.1364824 1.1364766 1.1361785 1.1362098 1.1361988 1.13617   1.1361839
 1.1361667 1.1361966 1.1361775 1.1361974 1.1358706 1.1358967 1.1358974
 1.135

In [167]:
opt_norm_Xs=norm_training_Xs.tolist()
opt_norm_ys=norm_training_ys.tolist()


In [168]:

print(len(norm_training_Xs.tolist()))
print(len(opt_norm_Xs))
print(len(opt_norm_ys))
print(len(choices))
print(len(values))


25
25
25
75
75


In [169]:
max_sample=[]
errs=[]
smerrs=[]
total_errs=[]

pred_points=[]
pred_vals=[]

for i in range((len(choices))+1):

    if i>0:
        opt_norm_Xs.append(choices[i-1])
        opt_norm_ys.append(values[i-1])

    gp_model = tfd.GaussianProcessRegressionModel(
        kernel=kernel,
        index_points=valid_norm_Xs,
        observation_index_points=opt_norm_Xs,
        observations=opt_norm_ys,
        observation_noise_variance=noise_var,
        predictive_noise_variance=0.,
        jitter=1e-4)
    pred_norm_ys = gp_model.mean().numpy()
    pred_real_ys = transform_y.inverse_transform(pred_norm_ys.reshape(-1,1)).flatten()

    valid_errs = pred_real_ys - valid_real_ys
    total_errs.append(valid_errs)
    errs.append(np.amax(valid_errs))
    sqvalid_errs = np.square(valid_errs)
    smerrs.append(np.sqrt(np.mean(sqvalid_errs)))
    max_sample.append(transform_y.inverse_transform(np.array([min(opt_norm_ys)])))
    
    # Predicted Maximum
    
    if i%3== 0:
        @tf.function(autograph=False, experimental_compile=False)
        def eval_model(x):
            gp_model = tfd.GaussianProcessRegressionModel(
                kernel=kernel,
                index_points=[x],
                observation_index_points=opt_norm_Xs,
                observations=opt_norm_ys,
                observation_noise_variance=noise_var,
                predictive_noise_variance=0.,
                jitter=1e-4)   
            
            return gp_model.mean()
##### GETTING MAX        
#         pred_point, pred_value = pso(func=lambda x: -eval_model(np.float32(x)), lb=norm_lb, ub=norm_ub, maxiter=25,
#                             debug=False)
#         norm_point = transform_X.inverse_transform([pred_point])[0].astype(np.float32)
#         norm_val = transform_y.inverse_transform([-pred_value])[0].astype(np.float32)
######

##### GETTING MIN        
#         pred_point, pred_value = pso(func=lambda x: eval_model(np.float32(x)), lb=norm_lb, ub=norm_ub, maxiter=25,
#                             debug=False)
#         norm_point = transform_X.inverse_transform([pred_point])[0].astype(np.float32)
#         norm_val = transform_y.inverse_transform([pred_value])[0].astype(np.float32)
######

#         pred_points.append(norm_point)
#         pred_vals.append(norm_val)
        
    ##
    
max_sample=np.array(max_sample)
max_sample=max_sample.flatten()


######## MAX
# filepath = os.path.join(save_path, 'predicted_optimums'+str(result_idx)+'.npyz')
# to_save = {
#     'values': pred_vals,
#     'choices': pred_points,
# }

####### MIN
# filepath = os.path.join(save_path, 'predicted_minimums'+str(result_idx)+'.npyz')
# to_save = {
#     'values': pred_vals,
#     'choices': pred_points,
# }


# with open(filepath, 'wb') as file:
#     np.savez(file, **to_save)
    

In [170]:
path = os.path.join(save_path,'max_err'+str(result_idx)+'.npy')
with open(path, 'wb') as f:
    np.save(f, errs)

# plt.figure(figsize=(12, 4))
# plt.plot(errs)
# plt.xlabel("Optimization iteration")
# plt.ylabel("Validation error")
# path = os.path.join(log_path, 'mean_regrets.png')
# plt.savefig(path)
# plt.show()
# plt.savefig('max_loss.pdf',format='pdf',dpi=1200)



In [171]:
path = os.path.join(save_path,'smerrs'+str(result_idx)+'.npy')
with open(path, 'wb') as f:
    np.save(f, smerrs)


# plt.figure(figsize=(12, 4))
# plt.plot(smerrs)
# plt.xlabel("Optimization iteration")
# plt.ylabel("Validation error")
# # path = os.path.join(log_path, 'mean_regrets.png')
# # plt.savefig(path)
# # plt.show()
# plt.savefig('sme.pdf',format='pdf',dpi=1200)

In [172]:
path = os.path.join(save_path,'max_sample'+str(result_idx)+'.npy')
with open(path, 'wb') as f:
    np.save(f, max_sample)

# plt.figure(figsize=(12, 4))
# plt.plot(max_sample)
# plt.xlabel("Optimization iteration")
# plt.ylabel("Max sample value")
# # path = os.path.join(log_path, 'mean_regrets.png')
# # plt.savefig(path)
# # plt.show()
# plt.savefig('max.pdf',format='pdf',dpi=1200)
# max_sample

In [173]:
result_idx=result_idx+1

In [3]:
%%javascript
// run the first 19 cells
var n
var i;
for (n=0; n<10; n++){
    for(i=2; i<18; i++) {
        Jupyter.notebook.execute_cells([i]);
    }

    // set the global in the 20th cell:
    Jupyter.notebook.execute_cells([18]);

}

<IPython.core.display.Javascript object>

In [206]:
filepath= os.path.join('.','BO_MIN_data','max_err4.npy')
with open(filepath, 'rb') as file:
    values = np.load(file)
    
print(values)
filepath= os.path.join('.','BO_MIN_data','smerrs4.npy')
with open(filepath, 'rb') as file:
    values = np.load(file)

print(values)

print(len(values))

[0.00041616 0.00030696 0.0003227  0.00033832 0.00033414 0.00033414
 0.00033414 0.0003618  0.00036728 0.00036991 0.0003705  0.00036991
 0.00037122 0.00037193 0.00037277 0.00037217 0.00037158 0.0003711
 0.00037086 0.00037026 0.00037003 0.00036991 0.00036943 0.00036824
 0.00036407 0.00036049 0.00035846 0.00035727 0.0003556  0.00035524
 0.00035453 0.00035441 0.00035429 0.00035453 0.00035453 0.00035465
 0.00035429 0.00035417 0.00035322 0.00035322 0.00035322 0.00035298
 0.00035286 0.00035286 0.00035286 0.00035286 0.00035262 0.00035262
 0.00035262 0.00035226 0.00035214 0.00035214 0.00035214 0.00035167
 0.00035155 0.00035167 0.00035155 0.00035167 0.00035167 0.00035107
 0.00035107 0.00035107 0.00035155 0.00035107 0.00035131 0.00035119
 0.00035119 0.00035107 0.00035119 0.00035381 0.00035357 0.00035369
 0.00035381 0.00035381 0.00035381 0.00035381]
[0.00041616 0.00030696 0.0003227  0.00033832 0.00033414 0.00033414
 0.00033414 0.0003618  0.00036728 0.00036991 0.0003705  0.00036991
 0.00037122 0.000

In [202]:
filepath= os.path.join('.','BO_data','max_err3.npy')
with open(filepath, 'rb') as file:
    values = np.load(file)


print(values)

print(len(values))

[0.00015533 0.00016367 0.00016403 0.00022602 0.00021946 0.00022113
 0.00023079 0.0002315  0.00023198 0.00022984 0.00023329 0.00024068
 0.00024199 0.0002408  0.00024056 0.00023651 0.00023973 0.00023687
 0.00023544 0.00023437 0.00022531 0.00022471 0.00022435 0.00022399
 0.00022244 0.00022173 0.00022101 0.00022089 0.00022018 0.00021935
 0.00021958 0.00021911 0.00021887 0.00021875 0.00021768 0.00021756
 0.00021911 0.00021887 0.00021291 0.00021255 0.00021267 0.00021243
 0.00021231 0.00020611 0.00020325 0.00020015 0.00019681 0.00020349
 0.00020349 0.00020361 0.00020206 0.0002017  0.00020087 0.00019979
 0.0001992  0.00019836 0.00019741 0.00019777 0.00019753 0.00019407
 0.00019348 0.00019348 0.00019193 0.00019038 0.00018954 0.00018883
 0.00018847 0.00018859 0.00018811 0.00018787 0.00018728 0.00018704
 0.00018597 0.00018513 0.00018501 0.00018847]
76


In [3]:
log_path = os.path.join('.','Uniform_data')
filepath = os.path.join(log_path,'samples.npyz')

with open(filepath, 'rb') as file:
    data = np.load(file)
    values = data['values']
    
len(values)

1000

In [7]:
values[0:25]

array([1.13549127, 1.13559665, 1.13633011, 1.13604062, 1.13621663,
       1.13674894, 1.13580586, 1.13576565, 1.13627605, 1.13572132,
       1.1357574 , 1.13580697, 1.13670155, 1.13669186, 1.13616053,
       1.13664467, 1.13549922, 1.13605184, 1.13587973, 1.13588408,
       1.13632889, 1.13676279, 1.13667152, 1.13597394, 1.13634622])

In [10]:
for i in range(10):
    maxs=[]
    mins=[]
    start=100*i
    for j in range(25,101,3):
        opt=max(values[start:start+j+1])
        maxs.append(opt)
        opt=min(values[start:start+j+1])
        mins.append(opt)
        
    path= os.path.join(log_path,'maximums'+str(i)+'.npy')
    with open(path, 'wb') as file:
        np.save(file,maxs)
    
    path= os.path.join(log_path,'minimums'+str(i)+'.npy')
    with open(path, 'wb') as file:
        np.save(file,mins)
        
        

    
